Testing
----------------

If you want to test your pipeline you can add test cases at any point in the jupyter notebook. You add a test case by creating a function named `test_pipeline(pass_events)` at any point in your jupyter notebook. In this case, we place it at the bottom

We can then pass events through the `pass_events` function and use "probes" to check intermediate values.

Tests will be automatically run before deployment but can also be run using the CLI.

Setup section 
-------------

Gets run once at launch

In [1]:
from bspump.jupyter import *
import bspump.kafka
import json

In [2]:
some_constant=3

In [3]:
@register_connection
def connection(app):
  return bspump.kafka.KafkaConnection(app, "KafkaConnection")

BitSwan BSPump version devel


In [9]:
# we define a sample event to test our pipeline.
event = b"""{"foo":"bap"}"""

We use `auto_pipeline` to mark the start of the *pipeline section*. We also specifiy the source and sink for our pipeline at this time.

Test section
------------

We create a dictionary of test events mapping each input event to a function that checks the expected output. The expected output is the list of events that get sent to the sink for a given input event.

You can use test probes to probe local variables at a given point in the pipeline.

You can use function based tests to inspect inside output events.

In [ ]:
test_events = {
    b"""{"foo":"aaa"}""": {
        "expect": [b'{"foo": "A   A   A"}'],
    },
    b"""{"foo":"aab"}""": {
        "expect": [b'{"foo": "B   A   A"}'],
        "probes": {
            "after-upper": (lambda plocals, pglobals: plocals["event"]["foo"], "AAB"),
        }
    },
    b"""{"foo":"cab"}""": {
        "inspect": (lambda events: events[0][9], ord("B")),
    },
}

In [5]:
auto_pipeline(
    source=lambda app, pipeline: bspump.kafka.KafkaSource(app, pipeline, "KafkaConnection"),
    sink=lambda app, pipeline: bspump.kafka.KafkaSink(app, pipeline, "KafkaConnection")
)

Pipeline section
----------------

Everything after this is rerun every time an event comes in. At run time, the `event` variable is automatically set with the value of the event that comes from the source.

We can do whatever transformations we please, and then, by setting `event` at the end of the notebook, the value of `event` will automatically be sent to the sink.

In [10]:
event = json.loads(event.decode("utf8"))
event

{'foo': 'bap'}

In [11]:
event["foo"] = event["foo"].upper()
event

{'foo': 'BAP'}

Test probes
-----------

If you want to examine intermediate values we can add test probes at any point in our pipeline. These test probes can be used in tests to examine any local variable.

In [ ]:
add_test_probe("after-upper")

In [12]:
event["foo"] = (" " * some_constant).join(reversed(list(event["foo"])))
event

{'foo': 'P   A   B'}

In [13]:
event = json.dumps(event).encode()
event

b'{"foo": "P   A   B"}'